In [1]:
import hail as hl

In [22]:
new_vcf=hl.read_matrix_table("/home/dpiscia/RDConnect/vcfLoader/output/7.0.0/annotatedVEPdbnSFPCaddClinvarExGnomad/variants19.ht")

In [23]:
new_vcf.count_rows()

654768

In [24]:
new_vcf=new_vcf.filter_rows(new_vcf.locus.position < 60659)

In [25]:
vcf=new_vcf.filter_entries((new_vcf.sample.gt=="0/1") | (new_vcf.sample.gt=="1/1") )

In [26]:
vcf =  vcf.annotate_rows(ref=vcf.alleles[0],
                       alt=vcf.alleles[1],
                       pos=vcf.locus.position,
                       indel=hl.cond((hl.len(vcf.alleles[0]) != (hl.len(vcf.alleles[1]))) | (hl.len(vcf.alleles[0]) != 1) | (hl.len(vcf.alleles[0]) != 1), True, False),
                       samples_germline=hl.filter(lambda x: (x.dp > 8) & (x.gq > 20),hl.agg.collect(vcf.sample)))

In [27]:
vcf=vcf.explode_rows(vcf.samples_germline).drop("sample").rows()

In [29]:
from pyspark.sql.functions import lit
variants = vcf.to_spark().withColumn("chrom",lit(19))
variants.write.format("org.elasticsearch.spark.sql").save("rdconnect_13_reduced/7.0.0", mode='append')

FatalError: TaskCompletionListenerException: Could not write all entries for bulk operation [287/287]. Error sample (first [5] error messages):
	Cannot add a value for field [samples_germline.sample] since one of the intermediate objects is mapped as a nested object: [samples_germline]
	Cannot add a value for field [samples_germline.sample] since one of the intermediate objects is mapped as a nested object: [samples_germline]
	Cannot add a value for field [samples_germline.sample] since one of the intermediate objects is mapped as a nested object: [samples_germline]
	Cannot add a value for field [samples_germline.sample] since one of the intermediate objects is mapped as a nested object: [samples_germline]
	Cannot add a value for field [samples_germline.sample] since one of the intermediate objects is mapped as a nested object: [samples_germline]
Bailing out...

Java stack trace:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most

In [86]:
new_vcf.count_rows()

9

In [95]:
new_vcf=new_vcf.explode_rows(new_vcf.effs)

In [96]:
new_vcf=new_vcf.filter_entries(new_vcf.sample.gt=='0/1')

In [ ]:
new_vcf.e

In [89]:
new_vcf.filter_cols(new_vcf.s=="EPR000831").entry.show(10)

2019-06-21 08:18:03 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'
2019-06-21 08:18:03 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']


+---------------+-------------+-------------+--------+-------+------+-----+--------------+
| locus         | alleles     | s           | MIN_DP |   MQ0 | PGT  | PID | PL           |
+---------------+-------------+-------------+--------+-------+------+-----+--------------+
| locus<GRCh37> | array<str>  | str         |  int32 | int32 | call | str | array<int32> |
+---------------+-------------+-------------+--------+-------+------+-----+--------------+
| 19:60251      | ["G","A"]   | "EPR000831" |     NA |    NA | NA   | NA  | [0,0,0]      |
| 19:60319      | ["C","T"]   | "EPR000831" |     NA |    NA | NA   | NA  | [0,0,0]      |
| 19:60354      | ["T","A"]   | "EPR000831" |     NA |    NA | NA   | NA  | [0,0,0]      |
| 19:60454      | ["A","C"]   | "EPR000831" |     NA |    NA | NA   | NA  | [0,0,0]      |
| 19:60462      | ["C","T"]   | "EPR000831" |     NA |    NA | NA   | NA  | [0,21,197]   |
| 19:60477      | ["ACC","A"] | "EPR000831" |     NA |    NA | NA   | NA  | [0,21,197]   |

In [73]:
new_vcf.count_rows()

54

In [75]:
new_vcf.distinct_by_row().count_rows()

9

In [97]:
new_vcf=new_vcf.distinct_by_row()
new_vcf=new_vcf.group_rows_by(new_vcf.effs.gene_name).aggregate(mac = hl.agg.counter(new_vcf.sample.sample))

In [100]:
new_vcf.filter_cols(new_vcf.s=="EPR000831").entry.show(10)

2019-06-21 08:27:09 Hail: INFO: Coerced sorted dataset


+-----------+-------------+----------------------+
| gene_name | s           | mac                  |
+-----------+-------------+----------------------+
| str       | str         | dict<str, int64>     |
+-----------+-------------+----------------------+
| "WASH5P"  | "EPR000831" | {"EPR000831":1,NA:8} |
+-----------+-------------+----------------------+



In [99]:
new_vcf.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'gene_name': str
----------------------------------------
Entry fields:
    'mac': dict<str, int64>
----------------------------------------
Column key: ['s']
Row key: ['gene_name']
----------------------------------------


In [18]:
new_vcf.show(3)

2019-06-20 16:32:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-06-20 16:33:04 Hail: INFO: Ordering unsorted dataset with network shuffle


gene_id,0.mac,1.mac,2.mac,3.mac,4.mac,5.mac,6.mac,7.mac,8.mac,9.mac,10.mac
str,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
"""""",104815,104815,104815,104815,104815,104815,104815,104815,104815,104815,104815
"""ENSG00000004776""",410,410,410,410,410,410,410,410,410,410,410
"""ENSG00000004777""",2517,2517,2517,2517,2517,2517,2517,2517,2517,2517,2517


In [19]:
new_vcf.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'gene_id': str
----------------------------------------
Entry fields:
    'mac': int64
----------------------------------------
Column key: ['s']
Row key: ['gene_id']
----------------------------------------
